Aspetti e considerazioni implementative sul progetto

# Log file

Per la gestione dei file log si può utilizzare logging direttamente da Python

In [1]:
import logging

### Es. 1

Un primo modo per gestire i log è il seguente.

Setto i parametri per gestire il file di log. Questi dovrebbero se unico in tutti i moduli di cui è composto il progetto, rimanere gli stessi per tutti.

Quindi in un altra classe se importata una chiamata alla logging.info('test') funzionerà e manterrà la configurazione di cui sopra anche in questo secondo modulo/classe/file.

In [2]:
def set_log():
    """ Set log system """
    logging.basicConfig(
        #filename='./log/mylog.log', 
        level=logging.DEBUG,
        format='%(asctime)s:%(levelname)s:%(message)s')

Con la chiamata basicConfig setto la configurazione dei log che mi interessa avere:

* filename= (se presente) dichiaro dove salvare i log generati, in questa maniera salva in coda i log generati e non sovrascrive il file

* filemode='w' se si vuole sovrascrivere i log a ogni esecuzione di programma

* level= imposta a che livello di visione avere i log: DEBUG - INFO - WARNIND - ecc.. settare un livello inferiore ne implica tutti i suoi superiori, INFO es. non comprende DEBUG (logging.debug() non verranno presi in considerazione) Per default è settato a WARNING quindi non rileva i INFO e DEBUG

* datefmt='%m/%d/%Y %I:%M:%S %p' per cambiare il formato time

* format= (se presente) setta il formato dei log che si desidera avere
    * %(asctime)s riporta il timer del log
    * %(levelname)s riporta il livello del log
    * %(message)s riporta il messaggio del log

In [3]:
set_log()
logging.info('Started')
# do something
logging.info('Finished')

2021-06-28 17:59:55,489:INFO:Started
2021-06-28 17:59:55,490:INFO:Finished


## Es. 2

Se si vuole invece settare dei log univoci per classe o modello, non è possibile usare quello di prima, in quanto l'esempio di prima è univoco in tutto, se poi specificassimo in una classe un config diverso, poi in base agli import ci sarebbero degli scaling di configurazione che potrebbero interfererire e risultare log di un modulo nel formato che non volevamo.

Una cosa utile per avere log diversi per modulo e riportare una cosa simile in tutti i moduli di interesse

In [4]:
import logging

logger = logging.getLogger(__name__) #modulo di riferimento corrente

def log():
    """Setto i parametri per gestire il file di log"""
    logger.setLevel(logging.WARNING)
    formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')
    # FileHandler: outputfile
    file_handler = logging.FileHandler('./test.log') #dove salvare i log
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

In questa maniera ora dovrò utilizzare non più logging, ma logger come riferimento che è specifico per questa configurazione

In [5]:
log()
logger.info("ciao") # non verrà considerato perché il level è warning
logger.warning("allerta") # verrà salvato su file nella config di nostra scelta
# ma viene stampatoa video (se invocato solo questo corrattamente) in cascata con il config 
# della funzione della cella sopra

2021-06-28 17:59:55,508:WARNING:allerta


Se vogliamo anche mostrarlo correttamente a video nel formato di nostro interesse

In [6]:
def log():
    """Setto i parametri per gestire il file di log"""
    logger.setLevel(logging.WARNING)
    formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

    # StreamHandler: console
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)
    logger.addHandler(stream_handler)

    # FileHandler: outputfile
    file_handler = logging.FileHandler('./test.log')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

In [7]:
log()
logger.info("ciao") # non verrà considerato perché il level è warning
logger.warning("allerta") # adesso viene stampato su console pure oltre che salvato

2021-06-28 17:59:55,526:WARNING:allerta


# Verbose command

Vogliamo mettere la possibilità di inserire dei comandi all'invocazione del progetto: in particolare il verbose

In [8]:
import argparse

def arg_parse():
    """Verifico la possibile chiamata verbose"""
    parser = argparse.ArgumentParser(description="Script che implementa metriche per il M.S.R. : tipo1 ,tipo2...")
    parser.add_argument("-v", "--verbose", help="Restituisce output verboso", action="store_true")
    args = parser.parse_args()
    return args.verbose

Questo è il verbose che è una scelta opzionale.

E' possibile scegliere e inserire anche delle opzioni obligatorie invece settando altri aspetti

In [9]:
def log(verbos):
    """Setto i parametri per gestire il file di log"""
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')
    if verbos:
        # StreamHandler: console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(formatter)
        logger.addHandler(stream_handler)

    # FileHandler: outputfile
    file_handler = logging.FileHandler('./test.log')
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)

In [10]:
verb = arg_parse()
log(verb)

usage: ipykernel_launcher.py [-h] [-v]
ipykernel_launcher.py: error: unrecognized arguments: -f /home/leo/.local/share/jupyter/runtime/kernel-69490e74-9aac-42ae-936c-048ae1c9b9d3.json


SystemExit: 2

/home/leo/Scrivania/Python Projects/MSR/venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In questa maniera per modificare il basicConfig()
Es. se si vuole formattare la data e orario

In [ ]:
def log(verbos):
    """Setto i parametri per gestire il file di log"""
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s',
                                 datefmt="%Y-%m-%d %H:%M:%S" # 2021-07-03 17:27:51,275
                                 #datefmt='%d/%m/%Y %I:%M:%S %p' # 03/07/2021 05:40:38 PM
                                 )
             ...   